<a href="https://colab.research.google.com/github/anruki/Rubik_Encription/blob/main/encription_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Modelo de cifrado empleando las permutaciones del cubo de Rubik

## Importación de las librerías de python necesarias

In [1]:
import numpy as np
import secrets

La biblioteca `numpy` de python proporciona estructuras como arrays, y funciones para operar con ellos.

- Ejemplo de uso:

In [2]:
# Creación de dos matrices utilizando NumPy
matriz_a = np.array([[1, 2, 3], [4, 5, 6]])
matriz_b = np.array([[7, 8, 9], [10, 11, 12]])

# Multiplicación matricial (producto punto)
producto_punto = np.dot(matriz_a, matriz_b.T)

La biblioteca `secrets` de python permite generar números aleatorios de forma segura (TODO: explicar aquí por qué)

- Ejemplo de uso:

In [3]:
# Generación de un número aleatorio seguro en el rango especificado
num_aleatorio = secrets.randbelow(100)  # Genera un número aleatorio entre 0 y 99 (inclusive)
print("Número aleatorio:", num_aleatorio)

# Generación de números aleatorios con un número específico de bits
num_aleatorio_bits = secrets.randbits(16)  # Genera un número aleatorio de 16 bits
print("Número aleatorio (16 bits):", num_aleatorio_bits)

Número aleatorio: 52
Número aleatorio (16 bits): 53534


## Transformación del mensaje a numérico

En nuestro caso, para ver fácilmente la operación del proceso de encriptación, vamos a pasar el mensaje en caracteres a un mensaje compuesto por números, cada letra del abecedario es representada por un valor numérico. Por ejemplo:

A: 0
B: 1
C:2

Además vamos a añadir un factor extra de aleatoridad para reforzar la seguridad del modelo de encriptación. Será una implementación del **cifrado César**  al traducir el mensaje (y después se hará la operación de encriptación mediante el producto de matrices).

**El cifrado César** consiste en cambiar cada letra de un texto por otra letra que se encuentra un número fijo de posiciones más adelante en el alfabeto. Por ejemplo, si el desplazamiento es 3, entonces A se convierte en D, B en E, C en F, y así sucesivamente.

Es un cifrado de sustitución monoalfabético, lo que significa que cada letra del texto original es reemplazada por otra letra del mismo alfabeto, y siempre se usa el mismo conjunto de reemplazos para todas las letras. Sin embargo, debido a su simplicidad, es muy fácil de descifrar utilizando técnicas de análisis de frecuencia o fuerza bruta. Es por ello, que será solo un complemento del cifrado basado en el cubo de Rubik.

In [4]:
def caracteres_a_numeros(texto,cesar):
    def caracter_a_numero(caracter,cesar):
        if not caracter.isalpha():
            return None  # Retornar None si no es un carácter alfabético
        return ord(caracter.lower()) - ord('a') + cesar

    # Convertir cada carácter del texto a su equivalente numérico y guardarlos en el array
    numeros = []
    for caracter in texto:
        numero = caracter_a_numero(caracter,cesar)
        if numero is not None:
            numeros.append(numero)

    return np.array(numeros)  # Convertir la lista a un array NumPy

Probamos un mensaje de ejemplo y llamamos a la función anterior.
Si queremos que no haya ninguna alteración, especificamos que la variable `cesar` sea 0.

In [5]:
# Cadena de texto de ejemplo
mensaje = 'abcde'

# Llamar a la función con el texto dado
mensaje_num = caracteres_a_numeros(mensaje,0)

print("Números obtenidos:", mensaje_num)

Números obtenidos: [0 1 2 3 4]


Cuando la variable `cesar` toma el valor 2, se produce una permutación de los valores numéricos del mensaje:

In [6]:
# Llamar a la función con el texto dado
mensaje_num = caracteres_a_numeros(mensaje,2)

print("Números obtenidos:", mensaje_num)

Números obtenidos: [2 3 4 5 6]


El usuario podrá especificar esta variable, pero siempre será un número en **módulo 26** ya que es el número de desplazamientos máximos que se pueden hacer en el alfabeto (si se hacen 26 desplazamientos, se vuelve a la posición inicial).

$ 26 = 0 (mod 26) $

$ 28 = 2 (mod 26) $

Para toda transformación en un proceso de encriptación simétrica, debe haber una transformación inversa que permita al receptor desencriptar el mensaje.

In [ ]:
def numeros_a_caracteres(numeros, cesar):
    def numero_a_caracter(numero):
        if numero == -1:
            return ''  # Retornar una cadena vacía si el número es -1
        elif 0 <= numero < 26:
            return chr(numero + ord('a') - cesar)
        else:
            return None  # Retornar None si el número está fuera del rango válido

    # Convertir cada número en el array a su equivalente alfabético y guardarlos en una lista
    caracteres = []
    for numero in numeros:
        caracter = numero_a_caracter(numero)
        if caracter is not None:
            caracteres.append(caracter)

    # Unir los caracteres en una cadena de texto
    return ''.join(caracteres)

Con el ejemplo de antes:

In [ ]:
# Llamar a la función con la serie de números dada
mensaje = numeros_a_cadena(mensaje_num,2)

print("El mensaje es:", mensaje)

## Creación de la función de encriptación simétrica

El mensaje en binario se le aplica a una matriz que representa las permutaciones del cubo de Rubik. Concretamente es una matriz con 2x3 dimensiones, para representar las 2 caras permutables del cubo y los 3 ejes `[x,y,z]` sobre los que se puede rotar cada cara. Para que la matriz sea cuadrada (necesario para la encriptación simétrica) se le añade una fila de unos.

Al tratarse de una estructura cúbica, trabajamos en modulo 4, ya que cada permutación, si se repite el mismo movimiento 4 veces, se vuelve a la posición inicial.

**Generación de la matriz en el cuerpo módulo 4**

Se utiliza:
 - La función `randbelow()` de la biblioteca `secrets` para generar números aleatorios **módulo 4**, es decir de 0 a 3.

- La función `.array()` de la biblioteca `numpy` para obtener una matriz de `3 filas` y `2 columnas`

La notación de llamada de la función secrets.randbelow(4) y np.array() sigue la convención de utilizar el punto para acceder a los métodos y atributos dentro de un módulo o biblioteca en Python.

In [9]:
def rubik_matrix():
    # Función que calcula una matriz de permutaciones aleatorias (con una fila de ceros agregada para que sea cuadrada)

    # Definición del número de filas y columnas
    filas = 3
    columnas = 3
    # Inicialización de la matriz
    matriz_permutaciones = []

    # Crear la matriz con números aleatorios
    matriz_permutaciones = np.array([[secrets.randbelow(4) for _ in range(columnas)] for _ in range(filas-1)])

    # Agregar una fila de unos al final de la matriz
    fila_ceros = np.ones((1, columnas), dtype=int)
    matriz_permutaciones = np.append(matriz_permutaciones, fila_ceros, axis=0)

    return matriz_permutaciones

In [10]:
# Crear la matriz con números aleatorios
matriz_permutaciones = rubik_matrix()

# Asegurar que la matriz tenga inversa (determinante no nulo)
while np.linalg.det(matriz_permutaciones) == 0:
    matriz_permutaciones = rubik_matrix()

print(matriz_permutaciones)

[[3 0 0]
 [3 3 1]
 [1 1 1]]


**Multiplicación de la matriz por el vector del mensaje**



En un cifrado simétrico como este, la matriz (nuestra `llave privada`) tomará valores diferentes cada vez que se encripte un mensaje para una gestión de mensajes más segura.

 Sin embargo, las dimensiones de la matriz de permutaciones siempre son las mismas, y no tienen por qué coincidir con las dimensiones del mensaje a encriptar, es por ello, que se ha de 'trocear' o hacer 'slicing' del mensaje para que independientemente de sus dimensiones, pueda ser multiplicado matricialmente con la matriz de permutaciones.

In [11]:
mensaje_num

array([2, 3, 4, 5, 6])

In [12]:
# cifrado_num = np.dot(matriz_permutaciones, resultado)
n = mensaje_num.size
# Si el tamaño del mensaje es impar, añado un -1 al final
while n%3 != 0:
  mensaje_num = np.append(mensaje_num,-1)
  n = mensaje_num.size
m = n//3
resultado = mensaje_num.reshape(m, 3)
print(resultado)
# hago slice del array mensaje
cifrado_num = []
for i in range(m):
  cifrado_num = cifrado_num + [np.dot(matriz_permutaciones, resultado[i])]

[[ 2  3  4]
 [ 5  6 -1]]


$Matriz\_permutaciones \times mensaje\_num = cifrado\_num$

In [13]:
cifrado_num

[array([ 6, 19,  9]), array([15, 32, 10])]

Pasamos a carácter de nuevo.



In [14]:
cifrado_char = numeros_a_caracteres(np.concatenate(cifrado_num),0)
print(cifrado_char)

gtjpk


El mensaje ha sido modificado 1º mediante un cifrado César y 2º mediante la multiplicación por la matriz de permutaciones de un cubo de Rubik.


## Proceso inverso de desencriptación

Para descifrar el mensaje, calculamos la inversa de la matriz transformación:

In [15]:
# Calcular la inversa
inversa = np.linalg.inv(matriz_permutaciones)
print(inversa)

[[ 0.33333333  0.          0.        ]
 [-0.33333333  0.5        -0.5       ]
 [ 0.         -0.5         1.5       ]]


In [16]:
descifrado_num = []
for i in range(len(cifrado_num)):
  descifrado_num = descifrado_num + [np.dot(inversa, cifrado_num[i])]
print(descifrado_num)

[array([2., 3., 4.]), array([ 5.,  6., -1.])]


In [17]:
for i in range(m):
  mensaje = np.append(mensaje,numeros_a_caracteres(descifrado_num[i].astype(int),2))
print(''.join(mensaje)) # Para unificar los strings en una sola cadena

abcdeabcde_


Y así obtenemos el mensaje inicial.



---



## Estandarización del proceso

Anteriormente se ha explicado paso por paso el proceso, añadiendo explicaciones con ejemplos para cada paso. Pero si se quisiese implementar este modelo de encriptación para cualquier mensaje introducido, se hará mediante las siguientes funciones que se encargan de:
- Generar la matriz de permutaciones y seleccionar un número de desplazamientos (clave secreta)
- Transformar entrada de texto a numérico
- Encriptar
- Desencriptar
- Transformar entrada numérica a texto

In [18]:
def rubik_matrix():
    # Función que calcula una matriz de permutaciones aleatorias (con una fila de ceros agregada para que sea cuadrada)

    # Definición del número de filas y columnas
    filas = 3
    columnas = 3
    # Inicialización de la matriz
    matriz_permutaciones = np.zeros((3, 3))
    while np.linalg.det(matriz_permutaciones) == 0:
      # Crear la matriz con números aleatorios
      matriz_permutaciones = np.array([[secrets.randbelow(4) for _ in range(columnas)] for _ in range(filas-1)])

      # Agregar una fila de unos al final de la matriz
      fila_ceros = np.ones((1, columnas), dtype=int)
      matriz_permutaciones = np.append(matriz_permutaciones, fila_ceros, axis=0)


    return matriz_permutaciones

In [19]:
def generar_claves():

  # Crear la matriz de permutaciones del cubo de rubik
  matriz_permutaciones = rubik_matrix()

  print('--------------------------------------------')
  print('CLAVE SECRETA: ')
  print('Matriz de permutaciones cubo de Rubik: ',matriz_permutaciones)
  print('--------------------------------------------')
  return matriz_permutaciones

In [20]:
def cadena_a_numeros(cadena):
    numeros = []
    for caracter in cadena:
        numero = ord(caracter) - ord('a')
        numeros.append(numero)
    return numeros

In [22]:
def numeros_a_cadena(numeros):
    cadena = ""
    for numero in numeros:
      if numero>=0:
          caracter = chr(numero + ord('a'))
          cadena += caracter
    return cadena


In [29]:
def encriptar(matriz_permutaciones):
  mensaje = input('Introduzca un mensaje: ')
  mensaje_num = np.array(cadena_a_numeros(mensaje))
  print('--------------------------------------------')
  print("MENSAJE EN FORMA NUMÉRICA: ", mensaje_num)
  n = mensaje_num.size
  # Si el tamaño del mensaje es impar, añado un -1 al final
  while n%3 != 0:
    mensaje_num = np.append(mensaje_num,-1)
    n = mensaje_num.size
  m = n//3
  resultado = mensaje_num.reshape(m, 3)
  print(resultado)
  # slice del array mensaje
  cifrado_num = []
  for elemento in resultado:
    cifrado_num = cifrado_num + [np.dot(matriz_permutaciones, elemento)]
  print('CIFRADO EN FORMA NUMÉRICA:: ', cifrado_num)
  print('CIFRADO EN FORMA ALFABÉTICA:: ', numeros_a_cadena(np.concatenate([array % 26 for array in cifrado_num])))
  print('--------------------------------------------')
  return cifrado_num

In [24]:
def desencriptar(cifrado_num,matriz_permutaciones):
  inversa = np.linalg.inv(matriz_permutaciones)
  print(matriz_permutaciones)
  descifrado_num = []
  mensaje = []
  for vector in cifrado_num:
    descifrado_num = np.dot(inversa, vector)
    mensaje = np.append(mensaje,numeros_a_cadena(np.round(descifrado_num).astype(int)))
  return ''.join(mensaje) # Para unificar los strings en una sola cadena

## Aplicación real

In [25]:
matriz_permutaciones= generar_claves()

--------------------------------------------
CLAVE SECRETA: 
Matriz de permutaciones cubo de Rubik:  [[3 3 2]
 [1 2 0]
 [1 1 1]]
--------------------------------------------


In [30]:
cifrado_num = encriptar(matriz_permutaciones)

Introduzca un mensaje: jimena
--------------------------------------------
MENSAJE EN FORMA NUMÉRICA:  [ 9  8 12  4 13  0]
[[ 9  8 12]
 [ 4 13  0]]
CIFRADO EN FORMA NUMÉRICA::  [array([75, 25, 29]), array([51, 30, 17])]
CIFRADO EN FORMA ALFABÉTICA::  xzdzer
--------------------------------------------


In [ ]:
mensaje = desencriptar(cifrado_num,matriz_permutaciones)
print('El mensaje original es: ',mensaje)